In [1]:
from google.colab import drive
drive.mount("/content/drive")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from tqdm.notebook import tqdm
tqdm.pandas()


Mounted at /content/drive


In [2]:
qsbio_file = "drive/MyDrive/OrlyPred/Data/QSbio_PiQSi_annotations_V6_2020.csv"
qsbio_df = pd.read_csv(qsbio_file, error_bad_lines=False, low_memory=False, skiprows=21)
qsbio_df = qsbio_df.drop(["Unnamed: 18", "Unnamed: 19", "Unnamed: 20", "Unnamed: 21"], axis=1)
qsbio_filt = qsbio_df[['code', 'nsub', 'corrected_nsub', 'homo']]


In [3]:
def code_mod(code_string):

    pdb_code = str(code_string.split("_")[0])
    modified_code = pdb_code
    return modified_code

**verification for embedding**

In [ ]:
qsbio_filt

,code,nsub,corrected_nsub,homo
0,100d_1,NaN,NaN,NaN
1,101d_1,NaN,NaN,NaN
2,101m_1,1,NaN,1
3,102d_1,NaN,NaN,NaN
4,102l_1,1,NaN,1
...,...,...,...,...
210675,9wga_1,2,NaN,1
210676,9wga_2,1,NaN,1
210677,9wga_3,2,NaN,1
210678,9xia_1,4,4,1


In [ ]:
qsbio_filt.isnull().groupby(['corrected_nsub','nsub']).size()


corrected_nsub  nsub 
False           False     76251
True            False    122219
                True      12210
dtype: int64

In [ ]:
qsbio_filt["nsub"].count()


198470

In [ ]:
qsbio_filt[qsbio_filt.corrected_nsub.notna() & (qsbio_filt.nsub.isnull())]

,code,nsub,corrected_nsub,homo


In [ ]:

210680 - 122219  
88461 - 76251
210680  - 198470

12210

In [ ]:
qsbio_filt[qsbio_filt.nsub.notna() & (qsbio_filt.corrected_nsub.isnull())]

,code,nsub,corrected_nsub,homo
2,101m_1,1,NaN,1
4,102l_1,1,NaN,1
5,102m_1,1,NaN,1
6,103l_1,1,NaN,1
7,103m_1,1,NaN,1
...,...,...,...,...
210670,9pti_1,1,NaN,1
210672,9rnt_1,1,NaN,1
210675,9wga_1,2,NaN,1
210676,9wga_2,1,NaN,1


In [ ]:
# To summarize the nan count:
# there are 134429 corrected_nsub nan
#           12210 nsub nan
# All the nsub nans are also nan in corrected


In [ ]:
qsbio_filt['corrected_nsub'].fillna(qsbio_filt['nsub'], inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [ ]:
qsbio_filt["corrected_nsub"].count()

198470

In [ ]:
qsbio_filt[qsbio_filt.nsub.notna() & (qsbio_filt.corrected_nsub.isnull())]

,code,nsub,corrected_nsub,homo


In [17]:
# FROM QSBIO_parsing_for_embeddings_Homomer *******************************
# read qsbio data into a df 
qsbio_df = pd.read_csv(qsbio_file, error_bad_lines=False, low_memory=False, skiprows=21)
qsbio_df = qsbio_df.drop(["Unnamed: 18", "Unnamed: 19", "Unnamed: 20", "Unnamed: 21"], axis=1)

# qsbio_df.head()

# keep only pdbID, subunits columns and homomer indication
# qsbio_filt = qsbio_df[['code', 'nsub', 'corrected_nsub', 'homo']]
qsbio_filt = qsbio_df.copy()

# remove rows with Nan values of nsub
# removes 12210 rows
qsbio_filt.dropna(subset=["nsub"], inplace=True)

#fill empty corrected_nsub with the val from nsub
qsbio_filt['corrected_nsub'].fillna(qsbio_filt['nsub'], inplace=True)

# create a new column with just pdb id
qsbio_filt['pdb_code'] = qsbio_filt['code'].apply(lambda x: code_mod(x))

# summarize how many instances are unique for each pdb code
# print(qsbio_filt.groupby('pdb_code').nunique())

# for each pdb code, retain only rows that have a unique nsub value
qsbio_uniq = qsbio_filt.drop_duplicates(subset=['pdb_code','nsub'],keep="last")

# #store relevant cols as ints
# qsbio_uniq["homo"] = pd.to_numeric(qsbio_uniq['homo'], errors='coerce', downcast="integer")
# qsbio_uniq["corrected_nsub"] = pd.to_numeric(qsbio_uniq['corrected_nsub'], errors='coerce', downcast="integer")
# qsbio_uniq["nsub"] = pd.to_numeric(qsbio_uniq['nsub'], errors='coerce', downcast="integer")

# # retain only homomers, i.e. rows where homo is 1 (from the documentation: '1' indicates homomers, otherwise heteromers)
# qsbio_homomer = qsbio_uniq[qsbio_uniq["homo"] == 1]

# print(qsbio_homomer.head(20))

In [ ]:
# summarize how many instances are unique for each pdb code
print(qsbio_filt.groupby('pdb_code').nunique())

# for each pdb code, retain only rows that have a unique nsub value
qsbio_uniq = qsbio_filt.drop_duplicates(subset=['pdb_code','nsub'],keep="last")

#store relevant cols as ints
qsbio_uniq["homo"] = pd.to_numeric(qsbio_uniq['homo'], errors='coerce', downcast="integer")
qsbio_uniq["corrected_nsub"] = pd.to_numeric(qsbio_uniq['corrected_nsub'], errors='coerce', downcast="integer")
qsbio_uniq["nsub"] = pd.to_numeric(qsbio_uniq['nsub'], errors='coerce', downcast="integer")

# retain only homomers, i.e. rows where homo is 1 (from the documentation: '1' indicates homomers, otherwise heteromers)
qsbio_homomer = qsbio_uniq[qsbio_uniq["homo"] == 1]

print(qsbio_homomer.head(20))

In [6]:
qsbio_filt.groupby(['pdb_code','corrected_nsub'], axis=0).nunique().shape
# 127655  unique pdbs

(134087, 17)

In [7]:
qsbio_filt.groupby('pdb_code').nunique().shape


(127655, 18)

In [ ]:
qsbio_filt[qsbio_filt["pdb_code"] == "9wga"]

In [14]:
# qsbio_uniq = qsbio_filt.drop_duplicates(subset=['pdb_code', "nsub"],keep="last")


In [16]:
qsbio_uniq.shape

(137690, 19)

In [ ]:
# qsbio_homomer = qsbio_uniq[qsbio_uniq["homo"] == 1]
qsbio_homomer

In [ ]:
qsbio_uniq["homo"] = pd.to_numeric(qsbio_uniq['homo'], errors='coerce', downcast="integer")
qsbio_uniq["corrected_nsub"] = pd.to_numeric(qsbio_uniq['corrected_nsub'], errors='coerce', downcast="integer")
qsbio_uniq["nsub"] = pd.to_numeric(qsbio_uniq['nsub'], errors='coerce', downcast="integer")


In [20]:
print(qsbio_uniq["homo"].count())
print(qsbio_uniq["corrected_nsub"].count())
print(qsbio_uniq["nsub"].count())


137652
137653
137652


In [ ]:
qsbio_uniq.dropna()

In [ ]:
qsbio_homomer.dropna()

In [ ]:
qsbio_homomer.groupby('homo').nunique().shape


(1, 18)

In [ ]:
qsbio_homomer.shape

(119129, 19)

In [22]:
print(qsbio_homomer.groupby("nsub").size().count(), "- number of different oligomeric assemblies")
print(qsbio_homomer.shape, "shape of homomeric qsbio")
qsbio_homomer.groupby("nsub").size().sort_values(ascending=False)

29 - number of different oligomeric assemblies
(119129, 19) shape of homomeric qsbio


nsub
1.0     61270
2.0     36897
4.0     10186
3.0      4386
6.0      3093
8.0      1033
5.0       624
12.0      552
24.0      403
10.0      311
7.0        99
14.0       87
16.0       72
11.0       29
9.0        23
60.0       16
18.0       10
13.0        8
20.0        8
48.0        6
15.0        5
32.0        2
34.0        2
36.0        2
17.0        1
28.0        1
30.0        1
39.0        1
42.0        1
dtype: int64

In [23]:
print(qsbio_homomer.groupby("corrected_nsub").size().count(), "- number of different oligomeric assemblies")
print(qsbio_homomer.shape, "shape of homomeric qsbio")
qsbio_homomer.groupby("corrected_nsub").size().sort_values(ascending=False)

26 - number of different oligomeric assemblies
(119129, 19) shape of homomeric qsbio


corrected_nsub
1.0     54897
2.0     41496
4.0     11127
3.0      4426
6.0      3381
8.0      1465
12.0      596
5.0       523
24.0      453
10.0      428
14.0      107
7.0        81
16.0       61
9.0        21
60.0       19
11.0       13
18.0       10
13.0        8
15.0        5
48.0        3
20.0        3
32.0        2
17.0        1
34.0        1
36.0        1
39.0        1
dtype: int64

In [ ]:
def compare_edited_to_original(row_input):
  # print(row_input)
  # print(row_input.iloc[0])
  code = row_input["code"]
  count = 0
  if int(qsbio_df[qsbio_df["code"] == code]["nsub"].iloc[0]) != int(row_input["nsub"]):
    count += 1
    print(count)
    print(code)
   


In [ ]:
# qsbio_homomer.progress_apply(compare_edited_to_original, axis=1)

In [ ]:
qsbio_df[qsbio_df["code"] == "9wga_3" ]["nsub"]
code = "9wga_3"

In [ ]:
qsbio_df[qsbio_df["code"] == code]["nsub"].iloc[0]

'2'

In [ ]:
def compare_edited_to_original_homo(row_input):
  code = row_input["code"]
  count = 0
  qsbio_df.dropna()
  if int(row_input["homo"]) != int(qsbio_df[qsbio_df["code"] == code]["homo"].iloc[0]):
  # if int(qsbio_df[qsbio_df["code"] == code]["corrected_nsub"].iloc[0]) != int(row_input["corrected_nsub"]):
    count += 1
    print(count)


In [ ]:
# qsbio_homomer.progress_apply(compare_edited_to_original_homo, axis=1)

In [ ]:
def compare_edited_to_original_corrected_nsub(row_input):
  code = row_input["code"]
  count = 0
  qsbio_df.dropna()
  if int(row_input["corrected_nsub"]) is not np.nan and int(qsbio_df[qsbio_df["code"] == code]["corrected_nsub"].iloc[0]) is not np.nan:
    if int(row_input["corrected_nsub"]) != int(qsbio_df[qsbio_df["code"] == code]["corrected_nsub"].iloc[0]):
    # if int(qsbio_df[qsbio_df["code"] == code]["corrected_nsub"].iloc[0]) != int(row_input["corrected_nsub"]):
      count += 1
      print(count)
  print("finished!")


In [ ]:
# qsbio_homomer.progress_apply(compare_edited_to_original_corrected_nsub, axis=1)
# doesnt work since there are nan values somewhere 

In [ ]:
qsbio_homomer['nsub'].compare(qsbio_homomer['corrected_nsub'])

,self,other
262,4.0,2.0
309,1.0,2.0
324,3.0,4.0
345,2.0,4.0
346,2.0,4.0
...,...,...
210526,1.0,2.0
210531,1.0,2.0
210601,1.0,2.0
210625,1.0,2.0


In [27]:
qsbio_homomer["QSBIO_err_prob"] = pd.to_numeric(qsbio_homomer['QSBIO_err_prob'], errors='coerce', downcast="integer")
qsbio_homomer["best_BU"] = pd.to_numeric(qsbio_homomer['best_BU'], errors='coerce', downcast="integer")


qsbio_df_conf = qsbio_homomer[qsbio_homomer["QSBIO_err_prob"] < 15]
qsbio_df_conf_plus = qsbio_df_conf[qsbio_df_conf["best_BU"] == 1]
print(qsbio_df_conf_plus.shape, "after filtering for high conf")
print(qsbio_homomer.shape, "without filtering")
qsbio_df_conf_plus['nsub'].compare(qsbio_df_conf_plus['corrected_nsub'])

# (58745, 19) after filtering for high conf
# (119129, 19) without filtering


(58745, 19) after filtering for high conf
(119129, 19) without filtering


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,self,other
345,2.0,4.0
346,2.0,4.0
538,1.0,2.0
638,2.0,4.0
758,1.0,2.0
...,...,...
210433,1.0,2.0
210509,1.0,2.0
210513,1.0,2.0
210601,1.0,2.0


In [26]:
print(qsbio_df_conf_plus.groupby("corrected_nsub").size().count(), "- number of different oligomeric assemblies")
print(qsbio_df_conf_plus.shape, "shape of homomeric qsbio")
qsbio_df_conf_plus.groupby("corrected_nsub").size().sort_values(ascending=False)
# type(qsbio_df_conf_plus.groupby("corrected_nsub").size())

19 - number of different oligomeric assemblies
(58745, 19) shape of homomeric qsbio


corrected_nsub
1.0     28603
2.0     19665
4.0      5259
3.0      2101
6.0      1409
8.0       634
12.0      295
24.0      280
5.0       245
10.0      148
14.0       46
7.0        28
16.0        9
60.0        9
9.0         7
11.0        3
13.0        2
15.0        1
18.0        1
dtype: int64

In [25]:
print(qsbio_df_conf_plus.groupby("nsub").size().count(), "- number of different oligomeric assemblies")
print(qsbio_df_conf_plus.shape, "shape of conf_plus")
qsbio_df_conf_plus.groupby("nsub").size().sort_values(ascending=False)

19 - number of different oligomeric assemblies
(58745, 19) shape of conf_plus


nsub
1.0     30364
2.0     18169
4.0      5166
3.0      2080
6.0      1395
8.0       501
12.0      295
24.0      280
5.0       242
10.0      148
14.0       46
7.0        27
16.0        9
60.0        9
9.0         7
11.0        3
13.0        2
15.0        1
18.0        1
dtype: int64

In [ ]:
qsbio_df_conf_plus[["nsub", "corrected_nsub"]].hist(bins=20, range=[1, 15])

In [ ]:
qsbio_homomer[["nsub", "corrected_nsub"]].hist(bins=20, range=[1, 15])

In [44]:
# Keep only rows were nsub and corrected_nsub agree
qsbio_final = qsbio_df_conf_plus[qsbio_df_conf_plus['corrected_nsub'] == qsbio_df_conf_plus['nsub']]

In [46]:
qsbio_final.shape

(56798, 19)